In [10]:
#Llama Tokenizer - Step 1
from transformers import MimiModel, AutoFeatureExtractor, AutoTokenizer
import torch
import numpy as np

class TextTokenizer:
    def __init__(self, name='Llama_tokenizer'):
        self.tokenizer = AutoTokenizer.from_pretrained(name, legacy=False)
        print("text vocab size", self.tokenizer.vocab_size)

    def encode(self, text: str):
        tokens = self.tokenizer.encode(text)
        return tokens

    def decode(self, tokens):
        return self.tokenizer.decode(tokens)
    
class MimiTokenizer:
    def __init__(self, device):    
        self.device = device
        self.model = MimiModel.from_pretrained("kyutai/mimi")
        self.model.to(device)
        self.model.eval()
        self.feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi", device=device)
        self.sampling_rate = self.feature_extractor.sampling_rate
        self.n_codebooks = 8
        self.vocab_size = 2048

    @torch.inference_mode()
    def encode(self, waveform):
        inputs = self.feature_extractor(raw_audio=waveform, 
                                        sampling_rate=self.sampling_rate, 
                                        return_tensors="pt").to(self.device)
            
        output = self.model.encode(inputs["input_values"], inputs["padding_mask"], num_quantizers=self.n_codebooks)
        tokens = output.audio_codes[0].cpu().numpy()
        return tokens

    def decode(self, tokens):
        assert len(tokens.shape) == 2
        tokens = torch.tensor(np.expand_dims(tokens, axis=0)).to(self.device)
        output = self.model.decode(tokens)
        waveform = output.audio_values.cpu()
        return waveform

2024-12-09 13:56:21.421986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733732781.442067  115641 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733732781.447540  115641 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 13:56:21.468376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
import json
tokenizer = TextTokenizer()
with open('/home/subhash/.cache/indri/lj_speech/annotation/metadata.jsonl') as f:
    for line in f:
        data = json.loads(line)
        text = data['raw_text']
        tokens = tokenizer.encode(text)
        print(text)
        print(tokens)
        print(tokenizer.decode(tokens))
        break

text vocab size 128000
Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition
[128000, 39628, 11, 304, 279, 1193, 5647, 449, 902, 584, 527, 520, 3118, 11920, 11, 44642, 505, 1455, 422, 539, 505, 682, 279, 19071, 323, 44948, 15609, 304, 279, 68033]
<|begin_of_text|>Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition


In [28]:
# Mimi Tokeniser - Step 2
import numpy as np
mimi_tokens = np.load("/home/subhash/.cache/indri/lj_speech/tokens/mimi/LJ040-0046.npy")
print(mimi_tokens.shape)
#print(mimi_tokens)
type(mimi_tokens)

(8, 100)


numpy.ndarray

In [26]:
# Weave Tokens, codebook offset - Step 3
def weave_tokens(tokens):
    result = []
    max_length = max(len(codebook) for codebook in tokens)
    
    for i in range(max_length):
        for codebook_index, codebook in enumerate(tokens):
            if i < len(codebook):
                offset = 2048 * codebook_index + 128000
                result.append(codebook[i] + offset)                 
    return np.array(result)

weave_audio = weave_tokens(mimi_tokens.tolist())
weave_audio

array([128995, 130325, 132834, 135350, 138081, 138819, 140988, 143449,
       128572, 131272, 133038, 135276, 136552, 139416, 141865, 142522,
       129117, 131597, 133406, 134146, 136322, 139289, 141923, 143985,
       129069, 131186, 133662, 135220, 136512, 139853, 141760, 142410,
       129620, 131843, 133121, 134814, 137436, 138387, 141084, 142444,
       128080, 131002, 133038, 135147, 137077, 139665, 141897, 142752,
       129134, 131203, 132924, 134668, 136548, 140002, 140409, 143451,
       128715, 130220, 133849, 135225, 136357, 139775, 140460, 143596,
       129650, 131937, 132126, 134545, 136933, 139077, 141937, 143864,
       129532, 131639, 132479, 134835, 137883, 138246, 141974, 143201,
       128937, 131824, 133411, 134384, 136248, 138246, 141565, 143949,
       129077, 130323, 132375, 134147, 138069, 139010, 141512, 142954,
       129352, 130386, 133860, 134608, 138148, 140177, 141688, 143239,
       128749, 130955, 133781, 135938, 137985, 140133, 141270, 143942,
      

In [14]:
import torch
import numpy as np
from transformers import AutoTokenizer

class TTSTokenizer:
    def __init__(self, text_tokenizer_name='tts_tokenizer', audio_tokenizer_name='tts_tokenizer'):
        self.text_tokenizer = AutoTokenizer.from_pretrained(text_tokenizer_name, legacy=False)
        self.audio_tokenizer = AutoTokenizer.from_pretrained(audio_tokenizer_name, legacy=False)

    def encode(self, input_data, add_special_tokens=True):
        if isinstance(input_data, str):
            encoded_tokens = self.text_tokenizer.encode(
                input_data, 
                return_tensors='pt', 
                add_special_tokens=add_special_tokens
            )
            return encoded_tokens
        elif isinstance(input_data, list) and all(isinstance(item, str) for item in input_data):
            encoded_tokens = self.audio_tokenizer.encode(
                input_data, 
                return_tensors='pt', 
                add_special_tokens=add_special_tokens
            )
            return encoded_tokens
        else:
            raise TypeError("Input must be a string or a list of strings")

    def decode(self, tokens):
        if not isinstance(tokens, torch.Tensor):
            raise TypeError("Input must be a torch tensor of tokens")
        
        try:
            decoded_text = self.text_tokenizer.decode(tokens)
            return decoded_text
        except:
            try:
                decoded_tokens = self.audio_tokenizer.decode(tokens)
                return torch.tensor(decoded_tokens)
            except:
                raise ValueError("Unable to decode the provided tokens")

In [15]:
tokenizer = TTSTokenizer(text_tokenizer_name='tts_tokenizer', audio_tokenizer_name='tts_tokenizer')
audio_decoding = tokenizer.decode(tokens=torch.tensor(weave_audio))
text_decoding = tokenizer.decode(tokens=torch.tensor([128000, 39628, 11, 304, 279, 1193, 5647, 449, 902, 584, 527, 520, 3118, 11920, 11, 44642, 505, 1455, 422, 539, 505, 682, 279, 19071, 323, 44948, 15609, 304, 279, 68033]))
print(text_decoding)
print(audio_decoding)

<|begin_of_text|>Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition
[aco_739][aco_2069][aco_4578][aco_7094][aco_9825][aco_10563][aco_12732][aco_15193][aco_316][aco_3016][aco_4782][aco_7020][aco_8296][aco_11160][aco_13609][aco_14266][aco_861][aco_3341][aco_5150][aco_5890][aco_8066][aco_11033][aco_13667][aco_15729][aco_813][aco_2930][aco_5406][aco_6964][aco_8256][aco_11597][aco_13504][aco_14154][aco_1364][aco_3587][aco_4865][aco_6558][aco_9180][aco_10131][aco_12828][aco_14188]<|reserved_special_token_72|>[aco_2746][aco_4782][aco_6891][aco_8821][aco_11409][aco_13641][aco_14496][aco_878][aco_2947][aco_4668][aco_6412][aco_8292][aco_11746][aco_12153][aco_15195][aco_459][aco_1964][aco_5593][aco_6969][aco_8101][aco_11519][aco_12204][aco_15340][aco_1394][aco_3681][aco_3870][aco_6289][aco_8677][aco_10821][aco_13681][aco_15608][aco_1276][aco_3383][aco_4223][aco_6579][aco_9627][aco_9990][aco_13718]

In [37]:
#Appending the tokens to a single sequence
#text_tokens, task_tokens, speaker_tokens, audio_start_tokens, audio_tokens, common_stop_token.
tokenizer = TTSTokenizer(text_tokenizer_name='tts_tokenizer', audio_tokenizer_name='tts_tokenizer')

MIMI = '[mimi]'
CONVERT = '[convert]'
CONTINUE = '[continue]'
DEFAULT_SPEAKER = '[spkr_unk]'
COMMON_STOP = '[stop]'

import torch

@torch.no_grad()
def append_tokens(text, audio_tokens, speaker=DEFAULT_SPEAKER):
    audio_tokens = torch.tensor(audio_tokens, dtype=torch.int32).clone().detach()
    text_tokens = torch.tensor(tokenizer.encode(text), dtype=torch.int32).view(-1).clone().detach()
    convert_tokens = torch.tensor(tokenizer.encode(CONVERT, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
    continue_tokens = torch.tensor(tokenizer.encode(CONTINUE, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
    speaker_tokens = torch.tensor(tokenizer.encode(speaker, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
    mimi_tokens = torch.tensor(tokenizer.encode(MIMI, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
    stop_tokens = torch.tensor(tokenizer.encode(COMMON_STOP, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
    
    result = torch.cat([
        text_tokens,
        convert_tokens,
        #continue_tokens,
        speaker_tokens,
        mimi_tokens,
        audio_tokens,
        stop_tokens
    ])
    
    return result

In [38]:
result = append_tokens("Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition", weave_audio)

/tmp/ipykernel_115641/1348805275.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tokens = torch.tensor(tokenizer.encode(text), dtype=torch.int32).view(-1).clone().detach()
/tmp/ipykernel_115641/1348805275.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  convert_tokens = torch.tensor(tokenizer.encode(CONVERT, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
/tmp/ipykernel_115641/1348805275.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  continue_tokens = torch.tensor(tokenizer.encode(CONTINUE, add_

In [39]:
result

tensor([128000,  39628,     11,    304,    279,   1193,   5647,    449,    902,
           584,    527,    520,   3118,  11920,     11,  44642,    505,   1455,
           422,    539,    505,    682,    279,  19071,    323,  44948,  15609,
           304,    279,  68033, 144642, 144645, 144641, 128995, 130325, 132834,
        135350, 138081, 138819, 140988, 143449, 128572, 131272, 133038, 135276,
        136552, 139416, 141865, 142522, 129117, 131597, 133406, 134146, 136322,
        139289, 141923, 143985, 129069, 131186, 133662, 135220, 136512, 139853,
        141760, 142410, 129620, 131843, 133121, 134814, 137436, 138387, 141084,
        142444, 128080, 131002, 133038, 135147, 137077, 139665, 141897, 142752,
        129134, 131203, 132924, 134668, 136548, 140002, 140409, 143451, 128715,
        130220, 133849, 135225, 136357, 139775, 140460, 143596, 129650, 131937,
        132126, 134545, 136933, 139077, 141937, 143864, 129532, 131639, 132479,
        134835, 137883, 138246, 141974, 

In [63]:
import os
import torch
import json
import numpy as np
CACHE_DIR = '/home/subhash/.cache/indri'
def load_tokens(dataset_dir):
    metadata_path = f"{CACHE_DIR}/{dataset_dir}/annotation/metadata.jsonl"
    tokens_dir = os.path.join(CACHE_DIR, dataset_dir, 'tokens', 'mimi')
    with open(metadata_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, 1):
            data = json.loads(line.strip())            
            file_path = os.path.join(tokens_dir, data['id'] + '.npy')
            
            audio_tokens = np.load(file_path)
            weave_audio = weave_tokens(audio_tokens.tolist())
            yield data['raw_text'], weave_audio, data['speaker_id']

In [67]:
dataset = 'lj_speech'
for raw_text, audio_tokens, speaker in load_tokens(dataset_dir=dataset):
    with open('allowed_speakers.jsonl', 'r', encoding='utf-8') as file:
        allowed_speakers = [json.loads(line.strip()) for line in file]
    entry = next((item for item in allowed_speakers if item['dataset'] == dataset and item['speaker'] == speaker), None)
    if entry:
        combined = entry['combined']
    else:
        combined = DEFAULT_SPEAKER
    result = append_tokens(raw_text, audio_tokens, speaker=combined)
    print(result)
    print(tokenizer.decode(result))
    break

tensor([128000,  39628,     11,  ..., 140557, 142493, 144644],
       dtype=torch.int32)
<|begin_of_text|>Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition[convert][spkr_unk][mimi][aco_1442][aco_3215][aco_4335][aco_6316][aco_8867][aco_11749][aco_14000][aco_15936][aco_1463][aco_3102][aco_4741][aco_6853][aco_9438][aco_11023][aco_12637][aco_15755][aco_12][aco_3327][aco_5098][aco_7331][aco_8508][aco_11952][aco_12220][aco_14360][aco_1133][aco_3558][aco_5358][aco_6739][aco_7940][aco_11184][aco_12999][aco_15792][aco_595][aco_2796][aco_5106][aco_7093][aco_8529][aco_11455][aco_13701][aco_15488][aco_1516][aco_2061][aco_5212][aco_6954][aco_8614][aco_10897][aco_13674][aco_14271]<|reserved_special_token_178|>[aco_2240][aco_4828][aco_6706][aco_8849][aco_10478][aco_12378][aco_15799][aco_1051][aco_3045][aco_4302][aco_7390][aco_8951][aco_11910][aco_12257][aco_15170][aco_1639][aco_2699][aco_4302][aco_60

/tmp/ipykernel_115641/1348805275.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tokens = torch.tensor(tokenizer.encode(text), dtype=torch.int32).view(-1).clone().detach()
/tmp/ipykernel_115641/1348805275.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  convert_tokens = torch.tensor(tokenizer.encode(CONVERT, add_special_tokens=False), dtype=torch.int32).view(-1).clone().detach()
/tmp/ipykernel_115641/1348805275.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  continue_tokens = torch.tensor(tokenizer.encode(CONTINUE, add_

In [ ]:
# Sample data (replace this with your actual data)


# Function to get combined ID based on dataset and speaker
def get_combined_id(data, dataset, speaker):
    entry = next((item for item in data if item['dataset'] == dataset and item['speaker'] == speaker), None)
    return entry['combined'] if entry else None

# Example usage
dataset_input = "mls_eng_10k"
speaker_input = "2156"
combined_id = get_combined_id(data, dataset_input, speaker_input)

print(combined_id)  # Output: [spkr_mls_eng_10k_2156]

In [69]:
import pickle

def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

pickle_file_path = 'tokens/lj_speech_tokens.pkl'  
data = load_pickle_file(pickle_file_path)
print(len(data))


/home/subhash/miniconda3/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


13100
